In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import GRU, Dense, Embedding, Input, concatenate
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# variable
words = [
    'come quickly', 'emergency', 'father', 'fever', 'good luck',
    'headache', 'hello', 'help', 'hi', 'hungry',
    'like', 'mother', 'mother_father', 'mother_mother', 'not ok',
    'quickly', 'sorry', 'tomorrow', 'yogurt'
]
data_per_word = 27
data_length = data_per_word * len(words)
timesteps = 600
dimensions = 3 * 40

In [3]:
def get_bone_pos(frame):
    finger_name = ['thumb', 'index', 'middle', 'ring', 'pinky']
#     coordinate = ['x', 'y', 'z']
    bone_pos = np.zeros([40, 3]) # [cooridinates x fingers]
#     finger_tip = {}
    if 'right' in frame['hands']:
        for idx, finger in enumerate(finger_name):
            bone_pos[idx, :] = np.array(frame['hands']['right']['fingers'][finger]['bones']['distal']['next_joint'])
            bone_pos[5 + idx, :] = np.array(frame['hands']['right']['fingers'][finger]['bones']['intermediate']['next_joint'])
            bone_pos[10 + idx, :] = np.array(frame['hands']['right']['fingers'][finger]['bones']['proximal']['next_joint'])
            bone_pos[15 + idx, :] = np.array(frame['hands']['right']['fingers'][finger]['bones']['metacarpal']['next_joint'])
    if 'left' in frame['hands']:
        for idx, finger in enumerate(finger_name):
            bone_pos[20 + idx, :] = np.array(frame['hands']['left']['fingers'][finger]['bones']['distal']['next_joint'])
            bone_pos[25 + idx, :] = np.array(frame['hands']['left']['fingers'][finger]['bones']['intermediate']['next_joint'])
            bone_pos[30 + idx, :] = np.array(frame['hands']['left']['fingers'][finger]['bones']['proximal']['next_joint'])
            bone_pos[35 + idx, :] = np.array(frame['hands']['left']['fingers'][finger]['bones']['metacarpal']['next_joint'])

    return bone_pos

In [4]:
import json
import pandas as pd
import numpy as np

In [7]:
x = np.zeros([data_length, timesteps, dimensions])
y = np.array([])
for word_no, word in enumerate(words):
    for data_no in range(data_per_word):
        try:
            with open("./record/{}/json_{}.txt".format(word, data_no)) as json_data:
                json_data = json.load(json_data)
        except Exception as s:
            print ('error log:', s)
            
        json_data()
        for frame_no, frame in enumerate(json_data):
            if not frame['hands']: #check if frame[hands] is null
                continue
            bones_pos = get_bone_pos(frame)
            for idx, bone_pos in enumerate(bones_pos):
                x[word_no * data_per_word + data_no, frame_no, idx * 3: idx * 3 + 3] = bone_pos
#             right_thumbs = get_finger_tip(frame)[0]
#             right_index = get_finger_tip(frame)[1]
#             right_middle = get_finger_tip(frame)[2]
#             right_ring = get_finger_tip(frame)[3]
#             right_pinky = get_finger_tip(frame)[4]
#             left_thumbs = get_finger_tip(frame)[5]
#             left_index = get_finger_tip(frame)[6]
#             left_middle = get_finger_tip(frame)[7]
#             left_ring = get_finger_tip(frame)[8]
#             left_pinky = get_finger_tip(frame)[9]
#             x[word_no * data_per_word + data_no, frame_no, 0:3] = right_thumbs
#             x[word_no * data_per_word + data_no, frame_no, 3:6] = right_index
#             x[word_no * data_per_word + data_no, frame_no, 6:9] = right_middle
#             x[word_no * data_per_word + data_no, frame_no, 9:12] = right_ring
#             x[word_no * data_per_word + data_no, frame_no, 12:15] = right_pinky
#             x[word_no * data_per_word + data_no, frame_no, 15:18] = left_thumbs
#             x[word_no * data_per_word + data_no, frame_no, 18:21] = left_index
#             x[word_no * data_per_word + data_no, frame_no, 21:24] = left_middle
#             x[word_no * data_per_word + data_no, frame_no, 24:27] = left_ring
#             x[word_no * data_per_word + data_no, frame_no, 27:30] = left_pinky

        y = np.append(y, word_no)

In [21]:
x_train = np.zeros([data_length * 2 / 3, timesteps, dimensions])
y_train = np.zeros([data_length * 2 / 3])
x_test = np.zeros([data_length / 3, timesteps, dimensions])
y_test = np.zeros([data_length / 3])
for idx in range(data_length):
    if idx % 3 == 2:
        x_test[idx / 3] = x[idx]
        y_test[idx / 3] = y[idx]
    else:
        x_train[idx - idx / 3] = x[idx]
        y_train[idx - idx / 3] = y[idx]

In [22]:
shuffle_train = np.arange(len(x_train))
np.random.shuffle(shuffle_train)
x_train_shuffle = np.zeros([data_length * 2 / 3, timesteps, dimensions])
y_train_shuffle = np.zeros([data_length * 2 / 3])
for idx, item in enumerate(shuffle_train):
    x_train_shuffle[idx] = x_train[item]
    y_train_shuffle[idx] = y_train[item]

In [23]:
# encoding label
Y_train_shuffle = np_utils.to_categorical(y_train_shuffle, len(words))
Y_test = np_utils.to_categorical(y_test, len(words))

In [28]:
fingers = Input(shape=(timesteps, dimensions), name='fingers')
fingers_layers = GRU(128)(fingers)
fingers_layers = Dense(128, activation='relu')(fingers_layers)
fingers_layers = Dense(128, activation='relu')(fingers_layers)
output_layer = Dense(len(words), activation='softmax')(fingers_layers)
model = Model(inputs=fingers, outputs=output_layer)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
model.fit(x_train_shuffle, Y_train_shuffle, epochs=10, batch_size=8)

Epoch 1/10
342/342 [==============================] - 19s 55ms/step - loss: 2.9463 - acc: 0.0292
Epoch 2/10
342/342 [==============================] - 18s 51ms/step - loss: 2.9456 - acc: 0.0322
Epoch 3/10
342/342 [==============================] - 17s 51ms/step - loss: 2.9453 - acc: 0.0263
Epoch 4/10
342/342 [==============================] - 17s 51ms/step - loss: 2.9450 - acc: 0.0526
Epoch 5/10
342/342 [==============================] - 18s 51ms/step - loss: 2.9451 - acc: 0.0205
Epoch 6/10
342/342 [==============================] - 18s 52ms/step - loss: 2.9453 - acc: 0.0322
Epoch 7/10
342/342 [==============================] - 18s 53ms/step - loss: 2.9450 - acc: 0.0439
Epoch 8/10
342/342 [==============================] - 19s 55ms/step - loss: 2.9450 - acc: 0.0322
Epoch 9/10
342/342 [==============================] - 18s 52ms/step - loss: 2.9451 - acc: 0.0351
Epoch 10/10
342/342 [==============================] - 18s 52ms/step - loss: 2.9449 - acc: 0.0468


In [30]:
score, acc = model.evaluate(x_test, Y_test)
print('Test score:', score)
print('Test accuracy:', acc)

171/171 [==============================] - 1s 7ms/step
('Test score:', 2.9444440944850094)
('Test accuracy:', 0.05263157894736842)


In [101]:
def build_model(data_length, label_length):
    right_thumb_x = Input(shape=(data_length, 1), name='right_thumb_x')
    right_thumb_y = Input(shape=(data_length, 1), name='right_thumb_y')
    right_thumb_z = Input(shape=(data_length, 1), name='right_thumb_z')
    right_index_y = Input(shape=(data_length, 1), name='right_index_y')
    right_index_x = Input(shape=(data_length, 1), name='right_index_x')
    right_index_z = Input(shape=(data_length, 1), name='right_index_z')
    right_middle_x = Input(shape=(data_length, 1), name='right_middle_x')
    right_middle_y = Input(shape=(data_length, 1), name='right_middle_y')
    right_middle_z = Input(shape=(data_length, 1), name='right_middle_z')
    right_ring_x = Input(shape=(data_length, 1), name='right_ring_x')
    right_ring_y = Input(shape=(data_length, 1), name='right_ring_y')
    right_ring_z = Input(shape=(data_length, 1), name='right_ring_z')
    right_pinky_x = Input(shape=(data_length, 1), name='right_pinky_x')
    right_pinky_y = Input(shape=(data_length, 1), name='right_pinky_y')
    right_pinky_z = Input(shape=(data_length, 1), name='right_pinky_z')
    left_thumb_x = Input(shape=(data_length, 1), name='left_thumb_x')
    left_thumb_y = Input(shape=(data_length, 1), name='left_thumb_y')
    left_thumb_z, = Input(shape=(data_length, 1), name='left_thumb_z')
    left_index_x = Input(shape=(data_length, 1), name='left_index_x')
    left_index_y = Input(shape=(data_length, 1), name='left_index_y')
    left_index_z, = Input(shape=(data_length, 1), name='left_index_z')
    left_middle_x = Input(shape=(data_length, 1), name='left_middle_x')
    left_middle_y = Input(shape=(data_length, 1), name='left_middle_y')
    left_middle_z = Input(shape=(data_length, 1), name='left_middle_z')
    left_ring_x = Input(shape=(data_length, 1), name='left_ring_x')
    left_ring_y = Input(shape=(data_length, 1), name='left_ring_y')
    left_ring_z = Input(shape=(data_length, 1), name='left_ring_z')
    left_pinky_x = Input(shape=(data_length, 1), name='left_pinky_x')
    left_pinky_y = Input(shape=(data_length, 1), name='left_pinky_y')
    left_pinky_z = Input(shape=(data_length, 1), name='left_pinky_z')

    right_thumb_x_layers = LSTM(16, return_sequences=False)(right_thumb_x)
    right_thumb_y_layers = LSTM(16, return_sequences=False)(right_thumb_y)
    right_thumb_z_layers = LSTM(16, return_sequences=False)(right_thumb_z)
    right_index_y_layers = LSTM(16, return_sequences=False)(right_index_y)
    right_index_x_layers = LSTM(16, return_sequences=False)(right_index_x)
    right_index_z_layers = LSTM(16, return_sequences=False)(right_index_z)
    right_middle_x_layers = LSTM(16, return_sequences=False)(right_middle_x)
    right_middle_y_layers = LSTM(16, return_sequences=False)(right_middle_y)
    right_middle_z_layers = LSTM(16, return_sequences=False)(right_middle_z)
    right_ring_x_layers = LSTM(16, return_sequences=False)(right_ring_x)
    right_ring_y_layers = LSTM(16, return_sequences=False)(right_ring_y)
    right_ring_z_layers = LSTM(16, return_sequences=False)(right_ring_z)
    right_pinky_x_layers = LSTM(16, return_sequences=False)(right_pinky_x)
    right_pinky_y_layers = LSTM(16, return_sequences=False)(right_pinky_y)
    right_pinky_z_layers = LSTM(16, return_sequences=False)(right_pinky_z)
    left_thumb_x_layers = LSTM(16, return_sequences=False)(left_thumb_x)
    left_thumb_y_layers = LSTM(16, return_sequences=False)(left_thumb_y)
    left_thumb_z_layers, =LSTM(16, return_sequences=False)(left_thumb_z)
    left_index_x_layers = LSTM(16, return_sequences=False)(left_index_x)
    left_index_y_layers = LSTM(16, return_sequences=False)(left_index_y)
    left_index_z_layers, =LSTM(16, return_sequences=False)(left_index_z)
    left_middle_x_layers = LSTM(16, return_sequences=False)(left_middle_x)
    left_middle_y_layers = LSTM(16, return_sequences=False)(left_middle_y)
    left_middle_z_layers = LSTM(16, return_sequences=False)(left_middle_z)
    left_ring_x_layers = LSTM(16, return_sequences=False)(left_ring_x)
    left_ring_y_layers = LSTM(16, return_sequences=False)(left_ring_y)
    left_ring_z_layers = LSTM(16, return_sequences=False)(left_ring_z)
    left_pinky_x_layers = LSTM(16, return_sequences=False)(left_pinky_x)
    left_pinky_y_layers = LSTM(16, return_sequences=False)(left_pinky_y)
    left_pinky_z_layers = LSTM(16, return_sequences=False)(left_pinky_z)

    output = concatenate(
        [
            right_thumb_x_layers,
            right_thumb_y_layers,
            right_thumb_z_layers,
            right_index_y_layers,
            right_index_x_layers,
            right_index_z_layers,
            right_middle_x_layers,
            right_middle_y_layers,
            right_middle_z_layers,
            right_ring_x_layers,
            right_ring_y_layers,
            right_ring_z_layers,
            right_pinky_x_layers,
            right_pinky_y_layers,
            right_pinky_z_layers,
            left_thumb_x_layers,
            left_thumb_y_layers,
            left_thumb_z_layers,
            left_index_x_layers,
            left_index_y_layers,
            left_index_z_layers,
            left_middle_x_layers,
            left_middle_y_layers,
            left_middle_z_layers,
            left_ring_x_layers,
            left_ring_y_layers,
            left_ring_z_layers,
            left_pinky_x_layers,
            left_pinky_y_layers,
            left_pinky_z_layers
        ]
    )

    output = Dense(label_length, activation='softmax', name='combine_input')(output)

    model = Model(
        inputs = [
            right_thumb_x,
            right_thumb_y,
            right_thumb_z,
            right_index_y,
            right_index_x,
            right_index_z,
            right_middle_x,
            right_middle_y,
            right_middle_z,
            right_ring_x,
            right_ring_y,
            right_ring_z,
            right_pinky_x,
            right_pinky_y,
            right_pinky_z,
            left_thumb_x,
            left_thumb_y,
            left_thumb_z,
            left_index_x,
            left_index_y,
            left_index_z,
            left_middle_x,
            left_middle_y,
            left_middle_z,
            left_ring_x,
            left_ring_y,
            left_ring_z,
            left_pinky_x,
            left_pinky_y,
            left_pinky_z
        ], 
        outputs = [
            output
        ]
    )

    model.compile(
        loss='categorical_crossentropy',
        optimizer='sgd',
        metrics=['accuracy']
    )

    return model

In [ ]:
rnn = build_model(93, 38)
rnn.fit(
    [
        features['right_thumb_x']
        features['right_thumb_y']
        features['right_thumb_z']
        features['right_index_y']
        features['right_index_x']
        features['right_index_z']
        features['right_middle_x']
        features['right_middle_y']
        features['right_middle_z']
        features['right_ring_x']
        features['right_ring_y']
        features['right_ring_z']
        features['right_pinky_x']
        features['right_pinky_y']
        features['right_pinky_z']
        features['left_thumb_x']
        features['left_thumb_y']
        features['left_thumb_z']
        features['left_index_x']
        features['left_index_y']
        features['left_index_z']
        features['left_middle_x']
        features['left_middle_y']
        features['left_middle_z']
        features['left_ring_x']
        features['left_ring_y']
        features['left_ring_z']
        features['left_pinky_x']
        features['left_pinky_y']
        features['left_pinky_z']
    ],
    [
        
    ]
)